# Reto 3: El problema final

Este reto consiste en predecir si una imagen contiene o no signos de retinopatía diabética (DR).
Para ello se proporciona un conjunto de datos que contiene características extraídas de imágenes.
Las características extraídas son:
+ [0] Evaluación de la calidad de la imagen , donde 0 = mala calidad 1 = calidad suficiente.
+ [1] El resultado binario de la evaluación previa, donde 1 indica una anormalidad retiniana severa y 0 su falta.
+ [2-7] Los resultados de la detección de microaneurismas (MA). Cada valor de característica representa el número de MAs encontrados en los niveles de confianza alfa = 0.5, ... , 1, respectivamente.
+ [8-15] contienen la misma información que [2-7] para los exudados. <br>
Sin embargo, ya que los exudados están representados por un conjunto de puntos en lugar del número de
píxeles de las lesiones, estas características se normalizan dividiendo los número de lesiones por  el diámetro de la ROI para compensar las diferentes tamaños de las imágenes.
+ [16] La distancia euclidiana del centro dela mácula y el centro del disco óptico para proporcionar información importante con respecto a la condición del paciente. Esta característica también se normaliza con el diámetro de la ROI
+ [17] El diámetro del disco óptico.
+ [18] El resultado binario de la clasificación basada en AM/FM.
+ [19] Etiqueta de clase. 1 = contiene signos de DR

El conjunto de datos, *retinopatia_reto3.csv*, consiste en 592 muestras de imágenes; cada uno de ellos representado por 19 características.

Para evaluar las propuestas se utilizará un conjunto de datos que se mantendrá oculto hasta después de la entrega

### Requisitos
+ **Se debe utilizar algún tipo de reducción de la dimensión o aprendizaje de variedades**
+ A continuación se debe entrenar un clasificador o combinación de clasificadores con las características transformadas según el requisito anterior.
+ Se debe entregar un cuaderno Jupyter con el nombre de los participantes.<br>
  *Por ejemplo*:   **Cuesta_Hinton.ipynb**
+ El cuaderno entregado debe seguir la estructura y reglas de este cuaderno

### Competición
+ Todos los cuadernos entregados se subirán al repo de GitHub y se ejecutarán en Binder, donde ya estará en conjunto de test que estaba oculto.
+ El resultado que se obtenga será la puntuación del reto.
+ **Importante** Es muy fácil asegurarte de que tu código funciona:
    1. Agrupa todo tu código en una única celda
    2. Copialo en el cuaderno del reto que hay en Binder
    3. Ejecuta el cuaderno 
    
### Plazo: lunes 16 de nov. a las 6 am.
Es decir, incluye toda la noche del domingo 15 de nov.
 
> "The final problem", A. C. Doyle (Strand Magazine, diciembre 1983), 
es el relato corto en el que Sherlock Holmes se enfrenta al Prof. Moriarty en las cataratas Reichenbach.

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [1]:
# NO TOCAR ESTA CELDA
# Conjunto distribuido para el reto

Challange_filename = '../../Datasets/retinopatia_reto3.csv'

In [2]:
# NO TOCAR ESTA CELDA
# El conjunto de test cambiará una vez se cierre la entrega
# Ahora mismo es el mismo conjunto que el de entrenamiento

Test_filename = '../../Datasets/retinopatia_test.csv' #<-- este nombre cambiará después del plazo de entrega

In [3]:
# NO TOCAR ESTA CELDA

#-[1]. Load data from CSV and put all in a single dataframe 'FullSet'

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#-[2]. Load data from CSV and put all in a single dataframe 'FullSet'

FullSet = pd.read_csv(Challange_filename, header=0)
FullX = FullSet.iloc[:,:-1]
FullY = FullSet.iloc[:,-1]

<table style="width:100%;"> 
 <tr style='background:lime'>
  <td style="text-align:left">
      <h2>Tu código debe empezar a partir de aquí y puede tener tantas celdas como quieras</h2>
      <p> Si quieres, puedes borrar (o convertir en RawNBConvert) las celdas de ejemplo
      <h3>Importante:</h3>
      <p>Tu código debe producir las siguientes variables: </p>
      <p> $\quad \bullet$ <b>clf:</b> el clasificador final con el que se realizará el test<br>
       $\quad \bullet$ <b>X_test:</b> el conjunto de test listo para ser usado por el método <b>predict</b><br>
       $\quad \bullet$ <b>Y_test:</b> es el vector de etiquetas del conjunto de X_test listo para ser usado por el método <b>confusion_matrix</b>
      </p>
  </td>
 </tr>
</table>

In [4]:
#-[4]. Split X_full into TRAIN and TEST in a single Stratified split

#valid_size = 0.3
#X_train, Y_train, X_test, Y_test = \
#   my.single_stratified_split( FullX,FullY, test_size=valid_size, random_state=seed)


nombres = ["Israel Peñalver Sánchez"]

#%%

#-- ejemplo de preprocesado --
X_train=FullX
Y_train=FullY


FullSet = pd.read_csv(Test_filename, header=0)
X_test = FullSet.iloc[:,:-1]
Y_test = FullSet.iloc[:,-1]



from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X_train)
Y = Y_train.values.ravel()




#%%
from sklearn.decomposition import PCA

n_components = 15
if n_components <= X.shape[1]:
    pca = PCA(n_components = n_components)
    pca.fit(X)
    X_proy = pca.transform(X)
else:
    print("ERROR: the number of princial components has to be less or equal than data dimension !")

X_test = scaler.transform(X_test)
X_test = pca.transform(X_test)
Y_test = Y_test.values.ravel()



clasifiers=[]
best=0
#-- ejemplo de entrenamiento --
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


C = [0.0001,0.001,0.01,0.1,1,5,9,10,11,15,100,1000,10000]
gamma = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]
params = [{'kernel': ['rbf'],'C' : C, 'gamma' : gamma}]
clf = GridSearchCV(SVC(random_state = seed),params,cv=4)
#clf = SVC(kernel='rbf', C=45, gamma=1, random_state = seed)
#clf.fit( X_proy, Y )
clasifiers.append((clf,X_proy,Y,X_test,Y_test))



clfOvR= OneVsRestClassifier(clf)
clasifiers.append((clfOvR,X_proy,Y,X_test,Y_test))


clfOvO= OneVsOneClassifier(clf)
clasifiers.append((clfOvO,X_proy,Y,X_test,Y_test))

lin_clf = LogisticRegression()
svm_clf = SVC(kernel='poly', degree=2, probability=True)
tree_clf = DecisionTreeClassifier(max_depth=2)
nbc_clf = GaussianNB()

voting_system = 'soft' #<-- choices are 'soft' , 'hard'
voting_clf = VotingClassifier(
    estimators=[('lin', lin_clf),  ('svc', svm_clf), ('tree', tree_clf), ('nbc',nbc_clf)],
    voting=voting_system)
clasifiers.append((voting_clf,X_proy,Y,X_test,Y_test))


n_estimators = 100
max_leaf_nodes = 16
max_depth = 1
RF_clf = RandomForestClassifier(n_estimators=n_estimators,
                                #max_depth=max_depth)
                                #max_leaf_nodes=max_leaf_nodes
                                )

clasifiers.append((RF_clf,X_proy,Y,X_test,Y_test))


from sklearn.ensemble import GradientBoostingClassifier

max_depth = 1
n_estimators = 100
learning_rate= 1

gb_clf = GradientBoostingClassifier(max_depth=max_depth,        \
                                    n_estimators=n_estimators,  \
                                    learning_rate=learning_rate)

clasifiers.append((gb_clf,X_proy,Y,X_test,Y_test))

'''RESULTADO: clf es el objeto con el clasificador'''

#%%

#-- ejemplo de test --


#-la evaluación se realiza en las celdas de abajo

for (clfi,X,Y,X_test,Y_test) in clasifiers:
    clfi.fit( X, Y )
    Y_hat = clfi.predict(X_test)
    conf_mat = confusion_matrix(Y_test , Y_hat)
    N_success  = np.trace(conf_mat)
    if N_success>best:
        clf=clfi
        best=N_success
        xDef=X_test

X_test=xDef
'''RESULTADO: X_test es el dataframe para utilizar en >>> Y_pred = clf.predict() 
   RESULTADO: Y_test es el array con las etiquetas para utilizar en >>> confusion_matrix(Y_test,Y_pred)
'''


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


'RESULTADO: X_test es el dataframe para utilizar en >>> Y_pred = clf.predict() \n   RESULTADO: Y_test es el array con las etiquetas para utilizar en >>> confusion_matrix(Y_test,Y_pred)\n'

<table style="width:100%;"> 
 <tr style='background:pink'>
  <td style="text-align:left">
      <h2>A partir de aquí ya no se pueden modificar las celdas</h2>
          <h3>Comprueba que:</h3>
          <p> $\quad \bullet$ tu clasificador está almacenado en la variable <b>clf</b><br>
              $\quad \bullet$ tienes el conjunto de test correctamente almacenado en la variable <b>X_test</b><br>
              $\quad \bullet$ tienes las etiquetas del conjunto de test correctamente almacenadas en la variable <b>Y_test</b><br>
          </p>
      
  </td>
 </tr>
</table>

## Test

In [5]:
# NO TOCAR ESTA CELDA

from sklearn.metrics import confusion_matrix

Y_hat = clf.predict(X_test)
conf_mat = confusion_matrix(Y_test , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_test.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Israel Peñalver Sánchez'] 

Confusion matrix:

[[446   0]
 [  0 505]] 

Outcome:

  :) HIT  = 951, (100.00%)
  :( FAIL = 0, (0.00%)
